In [1]:
# Install dependencies
!pip install gradio plotly pandas numpy

import gradio as gr
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

# Configuration with corrected P&L structure
BASELINE_PL = {
    "Revenue": 94.4,  # billion CHF
    "Cost_of_Goods_Sold": 50.2,
    "Gross_Profit": 44.2,  # Revenue - COGS
    "Operating_Expenses": 28.1,
    "Earnings_Before_Tax": 16.1,  # Gross Profit - Operating Expenses
    "Tax": 6.9,  # Estimated tax expense
    "Net_Income": 9.2,  # EBT - Tax
}

COGS_BREAKDOWN = {
    "Raw_Materials": 0.35,
    "Manufacturing": 0.45,
    "Energy": 0.20,
}

EMISSIONS_DATA = {
    "scope_1_2_emissions": 4.2,  # million tons CO2
}

IMPACT_COEFFICIENTS = {
    "carbon_tax_on_cogs": 0.6,
    "carbon_tax_on_opex": 0.4,
    "supply_chain_on_cogs": 0.7,
    "supply_chain_on_opex": 0.3,
    "raw_material_on_cogs": 1.0,
    "energy_on_cogs": 1.0,
}

# Calculate actual tax rate from baseline data
TAX_RATE = BASELINE_PL['Tax'] / BASELINE_PL['Earnings_Before_Tax']  # 6.9 / 16.1 = 0.429 (42.9%)

class ClimatePLModel:
    def __init__(self):
        self.baseline_pl = BASELINE_PL.copy()
        self.cogs_breakdown = COGS_BREAKDOWN.copy()
        self.emissions_data = EMISSIONS_DATA.copy()
        self.impact_coeffs = IMPACT_COEFFICIENTS.copy()
        self.tax_rate = TAX_RATE

    def calculate_climate_impact(self, carbon_tax, supply_chain_disruption,
                               raw_material_increase, energy_increase):
        impacted_pl = self.baseline_pl.copy()

        # Carbon tax impact
        total_emissions = self.emissions_data['scope_1_2_emissions']
        total_carbon_cost = (carbon_tax * total_emissions) / 1000
        cogs_impact = total_carbon_cost * self.impact_coeffs['carbon_tax_on_cogs']
        opex_impact = total_carbon_cost * self.impact_coeffs['carbon_tax_on_opex']

        # Supply chain impact
        supply_cogs_impact = (self.baseline_pl['Cost_of_Goods_Sold'] *
                             supply_chain_disruption / 100 *
                             self.impact_coeffs['supply_chain_on_cogs'])
        supply_opex_impact = (self.baseline_pl['Operating_Expenses'] *
                            supply_chain_disruption / 100 *
                            self.impact_coeffs['supply_chain_on_opex'])

        # Raw material impact
        raw_material_cogs = (self.baseline_pl['Cost_of_Goods_Sold'] *
                           self.cogs_breakdown['Raw_Materials'])
        raw_material_impact = raw_material_cogs * raw_material_increase / 100

        # Energy impact
        energy_cogs = (self.baseline_pl['Cost_of_Goods_Sold'] *
                      self.cogs_breakdown['Energy'])
        energy_impact = energy_cogs * energy_increase / 100

        # Apply impacts
        total_cogs_impact = cogs_impact + supply_cogs_impact + raw_material_impact + energy_impact
        total_opex_impact = opex_impact + supply_opex_impact

        # Update P&L with proper structure
        impacted_pl['Cost_of_Goods_Sold'] += total_cogs_impact
        impacted_pl['Operating_Expenses'] += total_opex_impact

        # Recalculate derived metrics following proper P&L structure
        impacted_pl['Gross_Profit'] = impacted_pl['Revenue'] - impacted_pl['Cost_of_Goods_Sold']
        impacted_pl['Earnings_Before_Tax'] = impacted_pl['Gross_Profit'] - impacted_pl['Operating_Expenses']

        # Calculate tax on new EBT using the actual baseline tax rate
        impacted_pl['Tax'] = impacted_pl['Earnings_Before_Tax'] * self.tax_rate
        impacted_pl['Net_Income'] = impacted_pl['Earnings_Before_Tax'] - impacted_pl['Tax']

        return impacted_pl

def create_waterfall_chart(baseline_pl, impacted_pl):
    baseline_net = baseline_pl['Net_Income']
    final_net = impacted_pl['Net_Income']
    total_impact = final_net - baseline_net

    fig = go.Figure(go.Waterfall(
        name="P&L Impact",
        orientation="v",
        measure=["absolute", "relative", "total"],
        x=["Baseline Net Income", "Climate Impact", "Final Net Income"],
        y=[baseline_net, total_impact, final_net],
        text=[f"{baseline_net:.2f}", f"{total_impact:.2f}", f"{final_net:.2f}"],
        textposition="outside",
        connector={"line": {"color": "rgb(63, 63, 63)"}},
        increasing={"marker": {"color": "green"}},
        decreasing={"marker": {"color": "red"}},
        totals={"marker": {"color": "blue"}}
    ))

    fig.update_layout(
        title="Climate Impact Waterfall Chart",
        xaxis_title="P&L Components",
        yaxis_title="Amount (billion CHF)",
        showlegend=False,
        height=500
    )
    return fig

def create_comparison_chart(baseline_pl, impacted_pl):
    # Define the order for proper P&L flow
    metrics_order = [
        "Revenue", "Cost_of_Goods_Sold", "Gross_Profit",
        "Operating_Expenses", "Earnings_Before_Tax", "Tax", "Net_Income"
    ]

    baseline_values = [baseline_pl[metric] for metric in metrics_order]
    impacted_values = [impacted_pl[metric] for metric in metrics_order]

    fig = go.Figure()
    fig.add_trace(go.Bar(
        name='Baseline',
        x=[metric.replace('_', ' ') for metric in metrics_order],
        y=baseline_values,
        marker_color='blue',
        opacity=0.7
    ))
    fig.add_trace(go.Bar(
        name='Climate Impact',
        x=[metric.replace('_', ' ') for metric in metrics_order],
        y=impacted_values,
        marker_color='red',
        opacity=0.7
    ))

    fig.update_layout(
        title="P&L Comparison: Baseline vs Climate Impact",
        xaxis_title="P&L Metrics",
        yaxis_title="Amount (billion CHF)",
        barmode='group',
        height=500
    )
    return fig

def create_margin_analysis_chart(baseline_pl, impacted_pl):
    """Create chart showing margin impacts"""
    baseline_gross_margin = (baseline_pl['Gross_Profit'] / baseline_pl['Revenue']) * 100
    baseline_ebt_margin = (baseline_pl['Earnings_Before_Tax'] / baseline_pl['Revenue']) * 100
    baseline_net_margin = (baseline_pl['Net_Income'] / baseline_pl['Revenue']) * 100

    impacted_gross_margin = (impacted_pl['Gross_Profit'] / impacted_pl['Revenue']) * 100
    impacted_ebt_margin = (impacted_pl['Earnings_Before_Tax'] / impacted_pl['Revenue']) * 100
    impacted_net_margin = (impacted_pl['Net_Income'] / impacted_pl['Revenue']) * 100

    margins = ['Gross Margin', 'EBT Margin', 'Net Margin']
    baseline_margins = [baseline_gross_margin, baseline_ebt_margin, baseline_net_margin]
    impacted_margins = [impacted_gross_margin, impacted_ebt_margin, impacted_net_margin]

    fig = go.Figure()
    fig.add_trace(go.Bar(
        name='Baseline',
        x=margins,
        y=baseline_margins,
        marker_color='blue',
        opacity=0.7
    ))
    fig.add_trace(go.Bar(
        name='Climate Impact',
        x=margins,
        y=impacted_margins,
        marker_color='red',
        opacity=0.7
    ))

    fig.update_layout(
        title="Margin Analysis: Baseline vs Climate Impact",
        xaxis_title="Margin Types",
        yaxis_title="Margin (%)",
        barmode='group',
        height=400
    )
    return fig

def update_interface(carbon_tax, supply_chain_disruption, raw_material_increase, energy_increase):
    model = ClimatePLModel()
    impacted_pl = model.calculate_climate_impact(
        carbon_tax, supply_chain_disruption, raw_material_increase, energy_increase
    )

    # Create comparison table with proper P&L order
    metrics_order = [
        "Revenue", "Cost_of_Goods_Sold", "Gross_Profit",
        "Operating_Expenses", "Earnings_Before_Tax", "Tax", "Net_Income"
    ]

    comparison_data = []
    for metric in metrics_order:
        baseline_value = BASELINE_PL[metric]
        impacted_value = impacted_pl[metric]
        change = impacted_value - baseline_value
        change_pct = (change / baseline_value * 100) if baseline_value != 0 else 0

        comparison_data.append({
            'Metric': metric.replace('_', ' '),
            'Baseline (billion CHF)': baseline_value,
            'Impacted (billion CHF)': impacted_value,
            'Change (billion CHF)': change,
            'Change (%)': change_pct
        })

    comparison_df = pd.DataFrame(comparison_data)

    # Create charts
    waterfall_chart = create_waterfall_chart(BASELINE_PL, impacted_pl)
    comparison_chart = create_comparison_chart(BASELINE_PL, impacted_pl)
    margin_chart = create_margin_analysis_chart(BASELINE_PL, impacted_pl)

    # Summary metrics
    net_income_change = ((impacted_pl['Net_Income'] - BASELINE_PL['Net_Income']) / BASELINE_PL['Net_Income']) * 100
    gross_margin_change = ((impacted_pl['Gross_Profit'] / impacted_pl['Revenue']) - (BASELINE_PL['Gross_Profit'] / BASELINE_PL['Revenue'])) * 100
    net_margin_change = ((impacted_pl['Net_Income'] / impacted_pl['Revenue']) - (BASELINE_PL['Net_Income'] / BASELINE_PL['Revenue'])) * 100

    total_climate_cost = (impacted_pl['Cost_of_Goods_Sold'] + impacted_pl['Operating_Expenses'] -
                         BASELINE_PL['Cost_of_Goods_Sold'] - BASELINE_PL['Operating_Expenses'])

    summary_text = f"""
    **Summary Impact:**
    - Net Income Change: {net_income_change:.2f}%
    - Gross Margin Change: {gross_margin_change:.2f} percentage points
    - Net Margin Change: {net_margin_change:.2f} percentage points
    - Total Climate Cost: {total_climate_cost:.2f} billion CHF
    - Tax Impact: {impacted_pl['Tax'] - BASELINE_PL['Tax']:.2f} billion CHF
    - Effective Tax Rate: {TAX_RATE*100:.1f}%
    """

    return comparison_df, waterfall_chart, comparison_chart, margin_chart, summary_text

# Create Gradio interface
with gr.Blocks(title="Nestlé Climate P&L Impact Model", theme=gr.themes.Soft()) as interface:
    gr.Markdown("""
    # 🌍 Nestlé Climate Change P&L Impact Model

    Explore how climate change factors affect Nestlé's financial performance.
    Adjust the parameters below to see real-time impact on the P&L statement.

    **P&L Structure:** Revenue → COGS → Gross Profit → Operating Expenses → Earnings Before Tax → Tax → Net Income
    """)

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### Climate Parameters")

            carbon_tax = gr.Slider(
                minimum=0, maximum=200, value=0, step=5,
                label="Carbon Tax ($/ton CO2)",
                info="Carbon pricing impact on operations"
            )

            supply_chain_disruption = gr.Slider(
                minimum=0, maximum=30, value=0, step=1,
                label="Supply Chain Disruption (%)",
                info="Logistics and procurement cost increases"
            )

            raw_material_increase = gr.Slider(
                minimum=0, maximum=50, value=0, step=1,
                label="Raw Material Cost Increase (%)",
                info="Agricultural input cost increases"
            )

            energy_increase = gr.Slider(
                minimum=0, maximum=100, value=0, step=1,
                label="Energy Cost Increase (%)",
                info="Manufacturing energy cost increases"
            )

            reset_btn = gr.Button("Reset to Baseline", variant="secondary")

            gr.Markdown("### Quick Scenarios")
            with gr.Row():
                conservative_btn = gr.Button("Conservative", size="sm")
                moderate_btn = gr.Button("Moderate", size="sm")
                severe_btn = gr.Button("Severe", size="sm")

        with gr.Column(scale=2):
            gr.Markdown("### P&L Impact Analysis")

            summary_text = gr.Markdown("Adjust parameters to see climate impact analysis.")

            with gr.Tabs():
                with gr.Tab("P&L Comparison Table"):
                    comparison_table = gr.Dataframe(
                        headers=["Metric", "Baseline (billion CHF)",
                               "Impacted (billion CHF)",
                               "Change (billion CHF)", "Change (%)"],
                        interactive=False
                    )

                with gr.Tab("Waterfall Chart"):
                    waterfall_chart = gr.Plot()

                with gr.Tab("P&L Comparison Chart"):
                    comparison_chart = gr.Plot()

                with gr.Tab("Margin Analysis"):
                    margin_chart = gr.Plot()

    # Event handlers
    def reset_values():
        return (0, 0, 0, 0) + update_interface(0, 0, 0, 0)

    def set_conservative():
        return (25, 5, 10, 15) + update_interface(25, 5, 10, 15)

    def set_moderate():
        return (75, 15, 25, 40) + update_interface(75, 15, 25, 40)

    def set_severe():
        return (150, 25, 40, 80) + update_interface(150, 25, 40, 80)

    # Connect events
    inputs = [carbon_tax, supply_chain_disruption, raw_material_increase, energy_increase]
    outputs = [comparison_table, waterfall_chart, comparison_chart, margin_chart, summary_text]

    for input_component in inputs:
        input_component.change(update_interface, inputs, outputs)

    reset_btn.click(reset_values, outputs=[carbon_tax, supply_chain_disruption,
                                         raw_material_increase, energy_increase] + outputs)

    conservative_btn.click(set_conservative, outputs=[carbon_tax, supply_chain_disruption,
                                                     raw_material_increase, energy_increase] + outputs)

    moderate_btn.click(set_moderate, outputs=[carbon_tax, supply_chain_disruption,
                                             raw_material_increase, energy_increase] + outputs)

    severe_btn.click(set_severe, outputs=[carbon_tax, supply_chain_disruption,
                                         raw_material_increase, energy_increase] + outputs)

    # Initial load
    interface.load(lambda: update_interface(0, 0, 0, 0), outputs=outputs)

    # Add comprehensive explanation section at the bottom
    gr.Markdown("""
    ---

    ## 📊 How Climate Impacts Affect P&L Items

    ### 🔥 Carbon Tax Impact
    **What it affects:** Cost of Goods Sold (60%) + Operating Expenses (40%)

    **Calculation:** Carbon Tax ($/ton) × Nestlé's Emissions (4.2M tons CO2) ÷ 1000

    **P&L Flow:**
    - ↑ Cost of Goods Sold → ↓ Gross Profit → ↓ Earnings Before Tax → ↓ Net Income
    - ↑ Operating Expenses → ↓ Earnings Before Tax → ↓ Net Income
    - Tax expense adjusts proportionally to new Earnings Before Tax

    **Example:** $75/ton carbon tax = $0.32B impact (60% to COGS, 40% to OpEx)

    ---

    ### 🚛 Supply Chain Disruption Impact
    **What it affects:** Cost of Goods Sold (70%) + Operating Expenses (30%)

    **Calculation:** Percentage × Baseline COGS/OpEx × Impact Coefficient

    **P&L Flow:**
    - ↑ Cost of Goods Sold → ↓ Gross Profit → ↓ Earnings Before Tax → ↓ Net Income
    - ↑ Operating Expenses → ↓ Earnings Before Tax → ↓ Net Income
    - Represents logistics delays, procurement challenges, transportation costs

    **Example:** 15% disruption = $5.3B impact on COGS + $0.8B on OpEx

    ---

    ### 🌾 Raw Material Cost Increase Impact
    **What it affects:** Cost of Goods Sold (100% - direct agricultural inputs)

    **Calculation:** Percentage × (COGS × 35% raw materials share)

    **P&L Flow:**
    - ↑ Cost of Goods Sold → ↓ Gross Profit → ↓ Earnings Before Tax → ↓ Net Income
    - Direct impact on agricultural commodities (coffee, cocoa, dairy, etc.)

    **Example:** 25% increase = $4.4B impact on COGS

    ---

    ### ⚡ Energy Cost Increase Impact
    **What it affects:** Cost of Goods Sold (100% - manufacturing energy)

    **Calculation:** Percentage × (COGS × 20% energy share)

    **P&L Flow:**
    - ↑ Cost of Goods Sold → ↓ Gross Profit → ↓ Earnings Before Tax → ↓ Net Income
    - Direct impact on manufacturing and production energy costs

    **Example:** 40% increase = $4.0B impact on COGS

    ---

    ### 📈 Cascading Effects Through P&L

    1. **Revenue:** Unchanged (assumes no price adjustments)
    2. **Cost of Goods Sold:** Increases from all climate factors
    3. **Gross Profit:** Decreases (Revenue - Higher COGS)
    4. **Operating Expenses:** Increases from carbon tax and supply chain disruption
    5. **Earnings Before Tax:** Decreases (Gross Profit - Higher OpEx)
    6. **Tax:** Adjusts to 42.9% of new Earnings Before Tax (based on Nestlé's actual tax rate)
    7. **Net Income:** Final impact after all cascading effects

    ### 🎯 Key Assumptions

    - **Tax Rate:** 42.9% effective tax rate (calculated from Nestlé's baseline data: 6.9B tax on 16.1B EBT)
    - **Revenue:** Held constant (no price pass-through modeled)
    - **Emissions:** Based on Nestlé's estimated Scope 1+2 emissions (4.2M tons CO2)
    - **COGS Breakdown:** 35% raw materials, 45% manufacturing, 20% energy
    - **Impact Distribution:** Based on industry analysis and climate risk studies

    ### 💡 Interpretation Tips

    - **Negative percentages** indicate profit reduction
    - **Margin compression** shows operational efficiency impact
    - **Tax impact** reflects the tax shield effect of lower profits
    - **Total climate cost** represents the sum of all direct cost increases
    - **When all parameters = 0**, impacted scenario matches baseline exactly
    """)

# Launch the interface
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://683ab8a290fdabe8a2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
